In [10]:
#imports
from rply import LexerGenerator
from rply.token import BaseBox

In [11]:
lg = LexerGenerator()

lg.add('NUMERO', r'\d+')
lg.add('MAIS', r'mais')
lg.add('MENOS', r'menos')
lg.add('MULTIPLICACAO', r'multiplicado_por')
lg.add('DIVISAO', r'dividido_por')
lg.add('ABREPARENTESES', r'\(')
lg.add('FECHAPARENTESES', r'\)')

lg.add('IGUAL', r'recebe')
lg.add('INTEIRO', r'inteiro')
lg.add('TEXTO', r'texto')
lg.add('PONTOVIRGULA', r'\;')

#adicionar if
lg.add('SE', r'se')
lg.add('FAZ', r'faz')
lg.add('CASO_CONTRARIO', r'caso_contrario_faz')
lg.add('ENQUANTO', r'enquanto')
lg.add('PARA_CADA', r'para_cada')
lg.add('COMPARADOR','for_igual_a')
lg.add('COMPARADOR','for_diferente_de')
lg.add('COMPARADOR','for_maior_ou_igual_a')
lg.add('COMPARADOR','for_menor_ou_igual_a')
lg.add('COMPARADOR','for_maior_que')
lg.add('COMPARADOR','for_menor_que')
lg.add('VARIAVEL', r'[a-zA-z][a-zA-z0-9]*')


lg.ignore('\s+')
lexer = lg.build()

In [12]:
tokens = lexer.lex("x gustavo y inteiro x recebe 10 y recebe 20")
for token in tokens:
    print(token)

Token('VARIAVEL', 'x')
Token('VARIAVEL', 'gustavo')
Token('VARIAVEL', 'y')
Token('INTEIRO', 'inteiro')
Token('VARIAVEL', 'x')
Token('IGUAL', 'recebe')
Token('NUMERO', '10')
Token('VARIAVEL', 'y')
Token('IGUAL', 'recebe')
Token('NUMERO', '20')


In [13]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITOR

class Programa(BaseBox):
    def __init__(self, declaracoes,atribuicoes):
        self.declaracoes = declaracoes
        self.atribuicoes = atribuicoes

    def accept(self, visitor):
        visitor.visit_programa(self)

class VarDeclaracoes(BaseBox):
    def __init__(self, declaracao,declaracoes):
        self.declaracao = declaracao
        self.declaracoes = declaracoes

    def accept(self, visitor):
        visitor.visit_vardeclaracoes(self)

class VarDeclaracao(BaseBox):
    def __init__(self, variavel,tp):
        self.variavel = variavel
        self.tp = tp

    def accept(self, visitor):
        visitor.visit_vardeclaracao(self)

class Atribuicoes(BaseBox):
    def __init__(self, atribuicao,atribuicoes):
        self.atribuicao = atribuicao
        self.atribuicoes = atribuicoes

    def accept(self, visitor):
        visitor.visit_atribuicoes(self)

class Atribuicao(BaseBox):
    def __init__(self,expr):
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atribuicao(self)

class Atrib(BaseBox):
    def __init__(self, variavel,expr):
        self.variavel = variavel
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class SeSenao(BaseBox):
    def __init__(self, expr1, comp, expr2, corpo1,corpo2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.corpo1=corpo1
        self.corpo2=corpo2

    def accept(self, visitor):
        visitor.visit_sesenao(self)


class Enquanto(BaseBox):
    def __init__(self, expr1, comp, expr2, corpo1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.corpo1=corpo1


    def accept(self, visitor):
        visitor.visit_enquanto(self)

class ParaCada(BaseBox):
    def __init__(self, variavelinic, exprinic, expr1, comp, expr2, variavelincr, exprincr, corpo1):
        self.variavelinic=variavelinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.variavelincr=variavelincr
        self.exprincr=exprincr
        self.corpo1=corpo1


    def accept(self, visitor):
        visitor.visit_paracada(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Variavel(Expr):
    def __init__(self, valor):
        self.valor = valor

class Numero(Expr):
    def __init__(self, valor):
        self.valor = valor


class OpBinario(Expr):
    def __init__(self, esquerda, direita):
        self.esquerda = esquerda
        self.direita = direita

class Soma(OpBinario):
  pass


class Subtracao(OpBinario):
  pass


class Multiplicacao(OpBinario):
  pass


class Divisao(OpBinario):
  pass

In [ ]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMERO', 'MAIS', 'MENOS', 'MULTIPLICACAO', 'DIVISAO', 'ABREPARENTESES', 'FECHAPARENTESES','IGUAL','INTEIRO','TEXTO','PONTOVIRGULA','SE','FAZ','CASO_CONTRARIO','ENQUANTO','PARA_CADA','COMPARADOR','VARIAVEL'],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('programa : vardecls statements')
def prog(p):
    return Programa(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "string")

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int")
